# 函数准备

In [2]:
import os
import numpy as np
import pandas as pd
import json
from copy import deepcopy
import re
import random
from random import choice
import eval
from mkdir import mk_dir
random.seed(10)
# -*- coding:utf-8 -*

# 训练集准备

In [10]:
# 判断病原菌
col_name = "是否是病原菌"
col_ab =  "pat"
cla_l = [": 是"]

# 判断免疫状态
col_name = "免疫状态"
col_ab =  "immu"
cla_l = [": 是"]

# 判断免疫状态
col_name = "预后"
col_ab =  "oc"
cla_l = [": 是"]

In [33]:
label_path = "../3.Orgnize/output/文献标注-校对/"
sped_l = [ele for ele in os.listdir(label_path) if 'add' in ele]
txt_path = "../2.Label_prepare/paper_ld"

for cla in cla_l:

    train_set = {
        "data": [{
            'paragraphs': [],
            'title': "LungB"
        }],
        "version": "LungB"
    }
    train_qa = []
    im = {"yes": False, "no": True}

    for sped in sped_l:

        # 读取标注信息
        df = pd.read_excel("%s/%s" % (label_path, sped), header=0)
        df = df.loc[df["是否是致病菌"] == "是", :]
        
        if col_name == "证据等级":
            df.loc[df[col_name] == cla, col_name] = "yes"
        else:
            df.loc[df[col_name] == cla.split("：")[0], col_name] = "yes"
        
        if col_name == "是否是致病菌":
            que_d = {"":'Can %s cause Pneumonia?' % df.loc[idx, "物种名称"]}
        if col_name == "免疫状态":
            df.loc[df[col_name] == "抑制", col_name] = "yes"
            que_d = {"":'Is patient immunosuppression?'}
        df.loc[df[col_name] != "yes", col_name] = "no"

        # 读取标题和摘要

        df["Full"] = 0

        for idx in df.index.tolist():

            with open("%s/%s/%s.txt" % (txt_path, sped.split("_")[0], df.loc[idx, "ID"]), "r") as f:
                txt = f.read()
            df.loc[idx, "Full"] = txt

            # 创建QA样本
            if df.loc[idx, col_name] in ['yes', 'no']:

                # im = (df.loc[idx, "是否是致病菌"] != 'yes')
                qa = {
                    'qas': [{
                        'id': '%s:%s' % (df.loc[idx, "ID"], df.loc[idx, "物种名称"]),
                        'question': que_d[cla.split("：")[0]],
                        'is_impossible': im[df.loc[idx, col_name]],
                        'answers': '%s' % df.loc[idx, col_name]
                    }],
                    'context':
                    '%s' % df.loc[idx, "Full"]
                }
                train_qa.append(qa)
        
        mk_dir("input/input_%s_yn%s" % (col_ab, cla.split("：")[0]))
        df.to_csv("input/input_%s_yn%s/%s.csv" %
                  (col_ab, cla.split("：")[0], sped.split("_")[0]), header=True, index=False)

    train_set['data'][0]["paragraphs"] = train_qa
    json.dump(train_set, open("input/input_%s_yn%s/train_set.json" %
              (col_ab, cla.split("：")[0]), "w"))